## RNN History

- RNN은 시퀀스 길이가 길어질 때, 앞쪽의 정보가 뒤쪽까지 효과적으로 전달되지 못하는 장기 의존성 문제(기울기 소실/폭주)를 가짐
- 이를 해결하기 위한 전략으로 제시된 것이 바로 게이트 매커니즘(Gate Mechanism)이며, 이를 통해 정보의 흐름을 제어하는 것이 가능해졌으며,  
  대표적인 게이트 매커니즘 모델이 LSTM과 GRU임

## LSTM (Long Short-Term Memory)

- LSTM은 '셀 상태(Cell State)'라는 별도의 정보 흐름 경로와 3개의 게이트(Forget, Input, Output)를 통해 장기 기억을 효과적으로 관리

### 1. 핵심 개념:

* **셀 상태 ($c_t$)**: 정보가 장기간 보존될 수 있는 '메모리 라인'. 게이트에 의해 정보가 추가되거나 제거됨.
* **은닉 상태 ($h_t$)**: 현재 시점의 출력을 만들고 다음 시점으로 전달되는 정보. 셀 상태를 가공하여 생성됨.
* **게이트**: Sigmoid 함수를 사용하여 0~1 사이의 값을 출력, 정보의 통과 비율을 조절.
    * **망각 게이트 ($f_t$)**: 과거 셀 상태($c_{t-1}$)에서 어떤 정보를 잊을지 결정.
    * **입력 게이트 ($i_t$)**: 현재 입력($x_t$)과 이전 은닉 상태($h_{t-1}$)를 바탕으로 어떤 새로운 정보($\tilde{c}_t$)를 셀 상태에 추가할지 결정.
    * **출력 게이트 ($o_t$)**: 업데이트된 셀 상태($c_t$)에서 어떤 정보를 현재 은닉 상태($h_t$)로 출력할지 결정.

### 2. 수식:

현재 시점 $t$의 입력 $x_t$, 이전 은닉 상태 $h_{t-1}$, 이전 셀 상태 $c_{t-1}$가 주어졌을 때:

* 망각 게이트: $f_t = \sigma(W_f [h_{t-1}, x_t] + b_f)$
* 입력 게이트: $i_t = \sigma(W_i [h_{t-1}, x_t] + b_i)$
* 셀 상태 후보: $\tilde{c}_t = \tanh(W_c [h_{t-1}, x_t] + b_c)$
* 셀 상태 업데이트: $c_t = f_t \odot c_{t-1} + i_t \odot \tilde{c}_t$
* 출력 게이트: $o_t = \sigma(W_o [h_{t-1}, x_t] + b_o)$
* 은닉 상태 업데이트: $h_t = o_t \odot \tanh(c_t)$

(표기: $\sigma$=Sigmoid, $\tanh$=Hyperbolic Tangent, $\odot$=원소별 곱셈, $[h_{t-1}, x_t]$=벡터 연결)


### 3. 코드

In [1]:
import torch
import torch.nn as nn
import math

class LSTMCellScratch(nn.Module):
    def __init__(self, input_size, hidden_size):
        """LSTM Cell 초기화 (PyTorch nn.Module 상속)"""
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # 가중치와 편향을 nn.Parameter로 등록 (학습 대상)
        # 입력과 이전 은닉 상태를 합친 크기
        concat_size = input_size + hidden_size

        # 각 게이트 및 셀 후보를 위한 하나의 큰 가중치 행렬과 편향 벡터
        # (계산 효율성을 위해 실제 라이브러리들은 이렇게 구현하는 경우가 많음)
        # 여기서는 이해를 돕기 위해 개별적으로 정의 (개념적 분리)
        # 망각 게이트 (Forget Gate)
        self.Wf = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bf = nn.Parameter(torch.Tensor(hidden_size))
        # 입력 게이트 (Input Gate)
        self.Wi = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bi = nn.Parameter(torch.Tensor(hidden_size))
        # 셀 상태 후보 (Candidate Cell State)
        self.Wc = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bc = nn.Parameter(torch.Tensor(hidden_size))
        # 출력 게이트 (Output Gate)
        self.Wo = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bo = nn.Parameter(torch.Tensor(hidden_size))

        self.reset_parameters()

    def reset_parameters(self):
        """가중치와 편향 초기화"""
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            # 모든 파라미터(Wf, bf, Wi, bi 등)에 대해 초기화 수행
             nn.init.uniform_(weight, -stdv, stdv)
        # 또는 nn.init.xavier_uniform_ 등 다른 초기화 방법 사용 가능

    def forward(self, xt, prev_state):
        """
        LSTM Cell의 단일 스텝 순전파

        Args:
            xt (torch.Tensor): 현재 시점 t의 입력 (batch_size, input_size)
            prev_state (tuple): 이전 시점의 상태 (h_{t-1}, c_{t-1})
                                h_{t-1}: (batch_size, hidden_size)
                                c_{t-1}: (batch_size, hidden_size)

        Returns:
            tuple: 현재 시점의 상태 (ht, ct)
                   ht: (batch_size, hidden_size)
                   ct: (batch_size, hidden_size)
        """
        prev_h, prev_c = prev_state

        # 1. 이전 은닉 상태와 현재 입력을 연결 (concatenate)
        # dim=1 : 피처(열) 방향으로 합침
        combined = torch.cat((prev_h, xt), dim=1)

        # 2. 망각 게이트 계산 (ft)
        ft = torch.sigmoid(torch.matmul(combined, self.Wf) + self.bf)

        # 3. 입력 게이트 계산 (it)
        it = torch.sigmoid(torch.matmul(combined, self.Wi) + self.bi)

        # 4. 셀 상태 후보 계산 (c_tilde_t)
        c_tilde_t = torch.tanh(torch.matmul(combined, self.Wc) + self.bc)

        # 5. 셀 상태 업데이트 (ct)
        ct = (ft * prev_c) + (it * c_tilde_t) # 원소별 곱셈

        # 6. 출력 게이트 계산 (ot)
        ot = torch.sigmoid(torch.matmul(combined, self.Wo) + self.bo)

        # 7. 은닉 상태 업데이트 (ht)
        ht = ot * torch.tanh(ct) # 원소별 곱셈

        return ht, ct

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


## GRU(Gated Recurrent Unit)
- GRU는 LSTM을 단순화한 구조로, 셀 상태 없이 은닉 상태만 사용하며 2개의 게이트(Reset, Update)로 정보 흐름을 제어

### 1. 핵심 개념:

* **은닉 상태 ($h_t$)**: 정보 저장과 출력을 모두 담당.
* **게이트**:
    * **리셋 게이트 ($r_t$)**: 과거 정보($h_{t-1}$) 중 현재와 관련 없는 정보를 얼마나 무시할지 결정하여 후보 은닉 상태($\tilde{h}_t$) 계산에 반영.
    * **업데이트 게이트 ($z_t$)**: 과거 정보($h_{t-1}$)와 현재 계산된 후보 정보($\tilde{h}_t$)를 어떤 비율로 조합하여 최종 은닉 상태($h_t$)를 만들지 결정. (LSTM의 망각+입력 게이트 역할 통합)

### 2. 수식:

현재 시점 $t$의 입력 $x_t$, 이전 은닉 상태 $h_{t-1}$가 주어졌을 때:

* 리셋 게이트: $r_t = \sigma(W_r [h_{t-1}, x_t] + b_r)$
* 업데이트 게이트: $z_t = \sigma(W_z [h_{t-1}, x_t] + b_z)$
* 후보 은닉 상태: $\tilde{h}_t = \tanh(W_h [r_t \odot h_{t-1}, x_t] + b_h)$
* 은닉 상태 업데이트: $h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$

### 3. 코드

In [2]:
import torch
import torch.nn as nn
import math

class GRUCellScratch(nn.Module):
    def __init__(self, input_size, hidden_size):
        """GRU Cell 초기화 (PyTorch nn.Module 상속)"""
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        concat_size = input_size + hidden_size

        # 리셋 게이트 (Reset Gate)
        self.Wr = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.br = nn.Parameter(torch.Tensor(hidden_size))
        # 업데이트 게이트 (Update Gate)
        self.Wz = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bz = nn.Parameter(torch.Tensor(hidden_size))
        # 후보 은닉 상태 (Candidate Hidden State)
        self.Wh = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bh = nn.Parameter(torch.Tensor(hidden_size))

        self.reset_parameters()

    def reset_parameters(self):
        """가중치와 편향 초기화"""
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
             nn.init.uniform_(weight, -stdv, stdv)

    def forward(self, xt, prev_h):
        """
        GRU Cell의 단일 스텝 순전파

        Args:
            xt (torch.Tensor): 현재 시점 t의 입력 (batch_size, input_size)
            prev_h (torch.Tensor): 이전 시점의 은닉 상태 h_{t-1} (batch_size, hidden_size)

        Returns:
            torch.Tensor: 현재 시점의 은닉 상태 ht (batch_size, hidden_size)
        """
        # 1. 이전 은닉 상태와 현재 입력을 연결
        combined = torch.cat((prev_h, xt), dim=1)

        # 2. 리셋 게이트 계산 (rt)
        rt = torch.sigmoid(torch.matmul(combined, self.Wr) + self.br)

        # 3. 업데이트 게이트 계산 (zt)
        zt = torch.sigmoid(torch.matmul(combined, self.Wz) + self.bz)

        # 4. 후보 은닉 상태 계산 (h_tilde_t)
        # 리셋 게이트를 적용한 이전 은닉 상태와 현재 입력을 연결
        combined_reset = torch.cat((rt * prev_h, xt), dim=1) # 원소별 곱셈
        h_tilde_t = torch.tanh(torch.matmul(combined_reset, self.Wh) + self.bh)

        # 5. 은닉 상태 업데이트 (ht)
        ht = (1 - zt) * prev_h + zt * h_tilde_t # 원소별 곱셈

        return ht

## Test #1

In [3]:
# 파라미터 설정
input_size = 10
hidden_size = 20
batch_size = 5

# 디바이스 설정 (GPU 사용 가능하면 GPU, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 입력 데이터 생성 (batch_size, input_size)
xt = torch.randn(batch_size, input_size).to(device)

# 초기 상태 생성 (batch_size, hidden_size)
h_prev = torch.zeros(batch_size, hidden_size).to(device)
c_prev = torch.zeros(batch_size, hidden_size).to(device) # LSTM만 필요

# --- LSTM Cell 테스트 ---
print("--- LSTM Cell Scratch Test ---")
lstm_cell_scratch = LSTMCellScratch(input_size, hidden_size).to(device)
# 모델 파라미터도 같은 디바이스로 이동
# lstm_cell_scratch.to(device) # 위 라인에서 .to(device)로 대체 가능

# 순전파
ht_lstm, ct_lstm = lstm_cell_scratch(xt, (h_prev, c_prev))

print(f"Input shape (xt): {xt.shape}")
print(f"Previous hidden state shape (h_prev): {h_prev.shape}")
print(f"Previous cell state shape (c_prev): {c_prev.shape}")
print(f"Output hidden state shape (ht_lstm): {ht_lstm.shape}")
print(f"Output cell state shape (ct_lstm): {ct_lstm.shape}")

# --- GRU Cell 테스트 ---
print("\n--- GRU Cell Scratch Test ---")
gru_cell_scratch = GRUCellScratch(input_size, hidden_size).to(device)
# gru_cell_scratch.to(device)

# 순전파
ht_gru = gru_cell_scratch(xt, h_prev)

print(f"Input shape (xt): {xt.shape}")
print(f"Previous hidden state shape (h_prev): {h_prev.shape}")
print(f"Output hidden state shape (ht_gru): {ht_gru.shape}")

# --- PyTorch 내장 Cell과 비교 ---
print("\n--- PyTorch Built-in Cell Comparison ---")
lstm_cell_builtin = nn.LSTMCell(input_size, hidden_size).to(device)
gru_cell_builtin = nn.GRUCell(input_size, hidden_size).to(device)

# 직접 구현한 Cell과 동일한 가중치로 설정해야 정확한 비교 가능
# 예: lstm_cell_builtin.weight_ih = lstm_cell_scratch.W_combined_input 등 (가중치 매핑 필요)

ht_lstm_builtin, ct_lstm_builtin = lstm_cell_builtin(xt, (h_prev, c_prev))
ht_gru_builtin = gru_cell_builtin(xt, h_prev)

print(f"Built-in LSTM output hidden shape: {ht_lstm_builtin.shape}")
print(f"Built-in GRU output hidden shape: {ht_gru_builtin.shape}")

--- LSTM Cell Scratch Test ---
Input shape (xt): torch.Size([5, 10])
Previous hidden state shape (h_prev): torch.Size([5, 20])
Previous cell state shape (c_prev): torch.Size([5, 20])
Output hidden state shape (ht_lstm): torch.Size([5, 20])
Output cell state shape (ct_lstm): torch.Size([5, 20])

--- GRU Cell Scratch Test ---
Input shape (xt): torch.Size([5, 10])
Previous hidden state shape (h_prev): torch.Size([5, 20])
Output hidden state shape (ht_gru): torch.Size([5, 20])

--- PyTorch Built-in Cell Comparison ---
Built-in LSTM output hidden shape: torch.Size([5, 20])
Built-in GRU output hidden shape: torch.Size([5, 20])


## Update
- 해당 논문('Were RNN all you need')에서 제시된 핵심 개념은 다음의 두 개임
    1. **단순화**: 핵심 기능을 유지하면서 게이트와 파라미터 수를 줄임
    2. **병렬화**: 순환적 업데이트 단계를 병렬 연관 스캔(parallel association scan) 연산으로 표현할 수 있도록 재구성해, 학습 및 추론 속드를 대폭 향상시킴  
                   이러한 방식은 Mamba 등의 영향을 받은 것으로 보임

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, Identity, Module
import math
import time
import numpy as np # 데이터 생성을 위해 필요

# --- 헬퍼 함수들 (minLSTM/minGRU 코드 및 부록 B 맥락 기반) ---
def exists(v):
    """값이 None이 아닌지 확인"""
    return v is not None

def default(v, d):
    """값이 존재하면 값을 반환하고, 아니면 기본값 d를 반환"""
    return v if exists(v) else d

# 안정적인 로그-누적합-지수 함수 (log-cumsum-exp)
def logcumsumexp(x, dim: int):
    """ numerically stable log-cumsum-exp
    from https://github.com/pytorch/pytorch/issues/31827#issuecomment-1027611157
    """
    # 최신 PyTorch 버전에는 torch.logcumsumexp 가 내장됨
    if hasattr(torch, "logcumsumexp"):
        #  print("내장 torch.logcumsumexp 사용")
         return torch.logcumsumexp(x, dim=dim)
    else:
         print("수동 logcumsumexp 사용")
         # 구버전 PyTorch를 위한 수동 구현
         max_val = torch.max(x, dim=dim, keepdim=True).values
         # max_val이 -inf일 때 (모든 요소가 -inf) 안정성 확보
         max_val = torch.where(torch.isinf(max_val), torch.zeros_like(max_val), max_val)
         x_adjusted = x - max_val
         cumulative_exp = torch.cumsum(x_adjusted.exp(), dim=dim)
         # 작은 값 클램핑으로 log(0) 가능성 처리 (또는 cumulative_exp 확인)
         log_cumulative_exp = torch.log(cumulative_exp.clamp(min=1e-38)) # 작은 값 클램핑
         return max_val + log_cumulative_exp

def heinsen_associative_scan_log(log_coeffs, log_values):
    """로그 공간에서의 연관 스캔 (Heinsen, Appendix B)"""
    # 구버전 PyTorch에서 필요할 수 있으므로 입력 텐서가 contiguous인지 확인
    log_coeffs = log_coeffs.contiguous()
    log_values = log_values.contiguous()

    # a*(t) = log_coeffs.cumsum(dim=1) = log(prod_{i=1...t} coeffs_i)
    a_star = log_coeffs.cumsum(dim = 1)

    # b*(t) = log_values(t) - a*(t) = log(values_t / prod_{i=1...t} coeffs_i)
    # log H0 + b*(t) = (log_values - a_star).logcumsumexp(dim = 1)
    # 여기서 H0는 초기 은닉 상태 항으로, 필요시 log_values 앞에 추가하여 효과적으로 처리
    log_h0_plus_b_star = logcumsumexp(log_values - a_star, dim = 1)

    # log h(t) = a*(t) + (log H0 + b*(t))
    log_h = a_star + log_h0_plus_b_star
    return log_h.exp() # exp()를 반환하여 양수 은닉 상태 얻기

# 사용자 정의 활성화 함수 g와 그 로그 버전 (Appendix B.3)
def g(x):
    """사용자 정의 활성화 함수 g"""
    return torch.where(x >= 0, x + 0.5, x.sigmoid())

def log_g(x):
    """사용자 정의 활성화 함수 g의 로그 버전"""
    # 로그 입력이 양수인지 확인
    relu_x_plus_0_5 = F.relu(x) + 0.5 # 항상 >= 0.5
    # sigmoid는 항상 > 0, softplus는 항상 > 0
    # log(sigmoid(x)) = -softplus(-x)
    return torch.where(x >= 0, relu_x_plus_0_5.log(), -F.softplus(-x))

## minLSTM (Minimal LSTM)


minLSTM은 스캔을 통한 병렬 실행을 위해 설계된 단순화된 LSTM 변형으로 간주되며. 출력 게이트를 생략하고 망각/입력 게이트 상호작용에 다른 공식을 사용할 수 있음.

### 1. 핵심 개념:

* **축소된 게이트:** 단일 선형 투사(linear projection)를 통해 망각($f_t$) 및 입력($i_t$) 게이트 관련 값과 은닉 후보($\tilde{h}_t$)만 명시적으로 계산합니다. 출력 게이트($o_t$)는 입력 투사로부터 직접 계산되지 않습니다.
* **암시적 출력:** 업데이트된 은닉 상태가 직접적으로 출력 역할을 하며, 선택적인 최종 투사(`to_out`)를 거칠 수 있습니다.
* **정규화된 업데이트 (순차 경로):** 단일 스텝 처리 시, 망각 게이트와 입력 게이트를 정규화하여($f'_t, i'_t$) 합이 1이 되도록 만들고, 이전 은닉 상태와 현재 후보 상태 사이의 선형 보간(linear interpolation)을 수행합니다: $h_t = h_{t-1} f'_{t} + \tilde{h}_t i'_{t}$. 여기서 $\tilde{h}_t$는 사용자 정의 활성화 함수 $g$를 사용하여 계산됩니다.
* **병렬 스캔 공식 (병렬 경로):** 시퀀스 처리 시, 게이트 및 은닉 후보 값들을 로그 공간(`log_f`, `log_i`, `log_tilde_h`)으로 변환하고, 연관 스캔 연산(`heinsen_associative_scan_log`)을 사용하여 모든 은닉 상태를 병렬로 계산합니다. 이는 표준 RNN의 순차적 의존성을 우회합니다.
* **활성화 함수 `g`:** 은닉 후보 계산에 사용자 정의 활성화 함수가 사용됩니다.

### 2. 수식:

결합된 투사를 위한 가중치 행렬을 $W_{hfi}$라고 가정합시다.
$ [\text{hidden}_t, \text{f\_gate}_t^{\text{raw}}, \text{i\_gate}_t^{\text{raw}}] = W_{hfi} x_t $

* 은닉 후보: $\tilde{h}_t = g(\text{hidden}_t)$
* 망각 게이트 (Sigmoid): $f_t = \sigma(\text{f\_gate}_t^{\text{raw}})$
* 입력 게이트 (Sigmoid): $i_t = \sigma(\text{i\_gate}_t^{\text{raw}})$
* 정규화된 망각 게이트: $f'_t = \frac{f_t}{f_t + i_t}$
* 정규화된 입력 게이트: $i'_t = \frac{i_t}{f_t + i_t}$
* 은닉 상태 업데이트: $h_t = h_{t-1} f'_{t} + \tilde{h}_t i'_{t}$ ($h_{t-1}$ 존재 시, 없으면 $h_t = \tilde{h}_t i'_{t}$)

*(참고: 병렬 경로는 스캔을 위해 `softplus`와 로그 공간 연산을 포함하는 다른 공식을 사용합니다.)*


###  3. 코드

In [5]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Identity, Module

class minLSTM(Module):
    def __init__(self, dim, expansion_factor = 1., proj_out = None):
        super().__init__()
        dim_inner = int(dim * expansion_factor) # 내부 차원 계산
        proj_out = default(proj_out, expansion_factor != 1.) # 출력 투사 여부 결정

        # 단일 선형 레이어가 입력 'x'를 hidden_candidate, raw_f_gate, raw_i_gate로 투사
        self.to_hidden_and_f_i_gate = Linear(dim, dim_inner * 3, bias = False)
        # 선택적 출력 투사 레이어
        self.to_out = Linear(dim_inner, dim, bias = False) if proj_out else Identity()

    def forward(self, x, prev_hidden = None, return_next_prev_hidden = False):
        seq_len = x.shape[1] # 시퀀스 길이 얻기

        # 입력 투사 후 세 부분으로 나누기
        hidden, f_gate_raw, i_gate_raw = self.to_hidden_and_f_i_gate(x).chunk(3, dim = -1)

        if seq_len == 1:
            # --- 순차 경로 (단일 타임 스텝) ---
            hidden_candidate = g(hidden)    # 사용자 정의 활성화 함수 g 적용
            f_gate = f_gate_raw.sigmoid() # Sigmoid 적용하여 망각 게이트 얻기
            i_gate = i_gate_raw.sigmoid() # Sigmoid 적용하여 입력 게이트 얻기

            # 게이트 정규화 (합이 1이 되도록 보장, 분모 0 방지 위해 작은 epsilon 추가)
            # 참고: f_gate + i_gate가 0이 될 수 있는 경우 안정성을 위해 작은 epsilon 필요
            f_gate_prime = f_gate / (f_gate + i_gate + 1e-8)
            i_gate_prime = i_gate / (f_gate + i_gate + 1e-8)

            # 정규화된 게이트를 사용한 선형 보간으로 은닉 상태 업데이트
            if exists(prev_hidden):
                out = (prev_hidden * f_gate_prime) + (hidden_candidate * i_gate_prime)
            else: # 초기 스텝 처리
                out = hidden_candidate * i_gate_prime
        else:
            # --- 병렬 경로 (스캔을 사용한 시퀀스 처리) ---
            # 망각 및 입력 영향에 대한 로그 공간 구성 요소 계산
            # diff = softplus(-f) - softplus(-i) = log(1+exp(-f)) - log(1+exp(-i)) = log((1+exp(-f))/(1+exp(-i)))
            diff = F.softplus(-f_gate_raw) - F.softplus(-i_gate_raw)
            # log_f = -softplus(diff) 는 안정적인 방식으로 log(f / (f+i))에 해당
            log_f = -F.softplus(diff)
            # log_i = -softplus(-diff) 는 안정적인 방식으로 log(i / (f+i))에 해당
            log_i = -F.softplus(-diff)

            # 사용자 정의 log_g를 사용하여 후보 은닉 상태의 로그 계산
            log_tilde_h = log_g(hidden)
            # 로그 입력 영향과 로그 후보 상태 결합
            log_values = log_i + log_tilde_h

            # 이전 은닉 상태가 제공된 경우 통합 (시퀀스 청킹/추론용)
            if exists(prev_hidden):
                # 일관성을 위해 log_g 사용? 논문/코드 맥락 필요.
                # minGRU 코드 구조상 단순 log 사용이 암시된 듯
                log_h_0 = prev_hidden.log() # prev_hidden이 양수라고 가정
                log_values = torch.cat((log_h_0, log_values), dim = 1)
                # 스캔을 위해 log_f 차원 정렬용 패딩
                log_f = F.pad(log_f, (0, 0, 1, 0), value=0.) # 시간 차원(dim 1) 패딩

            # 로그 공간에서 병렬 연관 스캔 적용
            # exp(scan_output) 반환, 양수 은닉 상태 제공
            out = heinsen_associative_scan_log(log_f, log_values)

            # prev_hidden이 앞에 추가된 경우 첫 번째 출력 스텝 제거
            if exists(prev_hidden):
                out = out[:, 1:] # 패딩이 달랐다면 조정 필요
            # 패딩 사용 시 원본 시퀀스 길이와 출력 길이 일치 확인 (필요시 재검토)
            # 원본 코드의 out = out[:, -seq_len:] 줄은 길이 변경 없다고 가정? 로직 재검토.
            # 여기서는 스캔이 입력 shape 기반으로 길이를 올바르게 처리한다고 가정.

        # 다음 호출을 위해 마지막 은닉 상태 저장
        next_prev_hidden = out[:, -1:]

        # 선택적 최종 투사 적용
        out = self.to_out(out)

        if not return_next_prev_hidden:
            return out
        return out, next_prev_hidden

## minGRU (Minimal GRU)

minGRU는 매우 단순화된 GRU로, 단일 게이트(업데이트 게이트와 유사)를 사용하고 병렬 스캔 메커니즘에 의존.  
표준 GRU의 리셋 게이트 기능은 생략되거나 단순화

### 1. 핵심 개념:

* **단일 게이트:** 단일 선형 투사로부터 은닉 후보($\tilde{h}_t$)와 함께 단 하나의 게이트 값(`gate`)만 명시적으로 계산합니다.
* **업데이트 게이트 역할:** 이 단일 `gate`는 주로 표준 GRU의 업데이트 게이트($z_t$)처럼 작동하여 이전 상태와 후보 상태 간의 보간을 제어합니다.
* **단순화된 리셋:** 리셋 게이트($r_t$)는 없거나 암시적으로 처리되는 것으로 보입니다 (예: 효과적으로 $r_t=1$). 후보 $\tilde{h}_t$는 $h_{t-1}$ 기반의 리셋 게이트에 의한 명시적 조절 없이 $g(\text{hidden}_t)$를 사용하여 직접 계산됩니다.
* **선형 보간 (순차 경로):** 업데이트는 직접적인 선형 보간(lerp)입니다: $h_t = (1 - z_t) h_{t-1} + z_t \tilde{h}_t$, 여기서 $z_t = \sigma(\text{gate}_t)$.
* **병렬 스캔 공식 (병렬 경로):** minLSTM과 유사하게 로그 공간 계산(`log_coeffs`, `log_z`, `log_tilde_h`)과 연관 스캔(`heinsen_associative_scan_log`)을 사용하여 병렬 시퀀스 처리를 수행합니다.
* **양수 은닉 상태:** 코드 주석에서 양수 은닉 상태를 강제한다고 언급하며, 이는 `exp()`를 출력하는 로그 공간 스캔을 통해 달성될 가능성이 높습니다.

### 2. 수식:

결합된 투사를 위한 가중치 행렬을 $W_{hg}$라고 가정합시다.
$ [\text{hidden}_t, \text{gate}_t^{\text{raw}}] = W_{hg} x_t $

* 은닉 후보: $\tilde{h}_t = g(\text{hidden}_t)$
* 업데이트 게이트 (Sigmoid): $z_t = \sigma(\text{gate}_t^{\text{raw}})$
* 은닉 상태 업데이트: $h_t = (1 - z_t) h_{t-1} + z_t \tilde{h}_t$ ($h_{t-1}$ 존재 시, 없으면 $h_t = z_t \tilde{h}_t$)

*(참고: 병렬 경로는 스캔을 위해 로그 공간 공식을 사용합니다.)*


### 3. 코드:

In [6]:
class minGRU(Module):
    def __init__(self, dim, expansion_factor = 1., proj_out = None):
        super().__init__()
        dim_inner = int(dim * expansion_factor) # 내부 차원 계산
        proj_out = default(proj_out, expansion_factor != 1.) # 출력 투사 여부 결정

        # 단일 선형 레이어가 입력 'x'를 hidden_candidate, raw_gate로 투사
        self.to_hidden_and_gate = Linear(dim, dim_inner * 2, bias = False)
        # 선택적 출력 투사 레이어
        self.to_out = Linear(dim_inner, dim, bias = False) if proj_out else Identity()

    def forward(self, x, prev_hidden = None, return_next_prev_hidden = False):
        seq_len = x.shape[1] # 시퀀스 길이 얻기

        # 입력 투사 후 두 부분으로 나누기
        hidden, gate_raw = self.to_hidden_and_gate(x).chunk(2, dim = -1)

        if seq_len == 1:
            # --- 순차 경로 (단일 타임 스텝) ---
            hidden_candidate = g(hidden) # 사용자 정의 활성화 함수 g 적용
            gate = gate_raw.sigmoid()    # Sigmoid 적용하여 업데이트 게이트 (z_t) 얻기

            # 선형 보간(lerp)을 사용하여 은닉 상태 업데이트
            if exists(prev_hidden):
                # out = (1 - gate) * prev_hidden + gate * hidden_candidate
                out = torch.lerp(prev_hidden, hidden_candidate, gate) # lerp 사용
            else: # 초기 스텝 처리
                out = hidden_candidate * gate
        else:
            # --- 병렬 경로 (스캔을 사용한 시퀀스 처리) ---
            # log_coeffs = -softplus(gate) 는 log(1-sigmoid(gate)) = log(1 - z_t)에 해당
            log_coeffs = -F.softplus(gate_raw) # (1 - z_t)의 로그

            # log_z = -softplus(-gate) 는 log(sigmoid(gate)) = log(z_t)에 해당
            log_z = -F.softplus(-gate_raw)    # z_t의 로그
            # 후보 은닉 상태의 로그 계산
            log_tilde_h = log_g(hidden)
            # 로그 업데이트 영향과 로그 후보 상태 결합 = log(z_t * h_tilde_t)
            log_values = log_z + log_tilde_h

            # 이전 은닉 상태가 제공된 경우 통합
            if exists(prev_hidden):
                 # prev_hidden이 양수라고 가정 (이전 스텝에서 강제됨)
                log_h_0 = prev_hidden.log()
                log_values = torch.cat((log_h_0, log_values), dim = 1)
                 # 스캔을 위해 log_coeffs 차원 정렬용 패딩
                log_coeffs = F.pad(log_coeffs, (0, 0, 1, 0), value=0.) # 시간 차원 패딩

            # 로그 공간에서 병렬 연관 스캔 적용
            # exp(scan_output) 반환, 양수 은닉 상태 강제
            out = heinsen_associative_scan_log(log_coeffs, log_values)

            # prev_hidden이 앞에 추가된 경우 출력 조정
            if exists(prev_hidden):
                 out = out[:, 1:] # 스캔 출력이 h_0의 영향을 포함한다고 가정
            # 올바른 길이 보장 (필요시 재검토)
            # out = out[:, -seq_len:]

        # 마지막 은닉 상태 저장
        next_prev_hidden = out[:, -1:]

        # 선택적 최종 투사 적용
        out = self.to_out(out)

        if not return_next_prev_hidden:
            return out
        return out, next_prev_hidden

## 비교 및 평가

In [7]:
# --- 이전 직접 구현 Cell (이전과 동일) ---
class LSTMCellScratch(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        concat_size = input_size + hidden_size
        self.Wf = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bf = nn.Parameter(torch.Tensor(hidden_size))
        self.Wi = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bi = nn.Parameter(torch.Tensor(hidden_size))
        self.Wc = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bc = nn.Parameter(torch.Tensor(hidden_size))
        self.Wo = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bo = nn.Parameter(torch.Tensor(hidden_size))
        self.reset_parameters()
    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
             nn.init.uniform_(weight, -stdv, stdv)
    def forward(self, xt, prev_state):
        prev_h, prev_c = prev_state
        combined = torch.cat((prev_h, xt), dim=1)
        ft = torch.sigmoid(torch.matmul(combined, self.Wf) + self.bf)
        it = torch.sigmoid(torch.matmul(combined, self.Wi) + self.bi)
        c_tilde_t = torch.tanh(torch.matmul(combined, self.Wc) + self.bc)
        ct = (ft * prev_c) + (it * c_tilde_t)
        ot = torch.sigmoid(torch.matmul(combined, self.Wo) + self.bo)
        ht = ot * torch.tanh(ct)
        return ht, ct

class GRUCellScratch(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        concat_size = input_size + hidden_size
        self.Wr = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.br = nn.Parameter(torch.Tensor(hidden_size))
        self.Wz = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bz = nn.Parameter(torch.Tensor(hidden_size))
        self.Wh = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bh = nn.Parameter(torch.Tensor(hidden_size))
        self.reset_parameters()
    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
             nn.init.uniform_(weight, -stdv, stdv)
    def forward(self, xt, prev_h):
        combined = torch.cat((prev_h, xt), dim=1)
        rt = torch.sigmoid(torch.matmul(combined, self.Wr) + self.br)
        zt = torch.sigmoid(torch.matmul(combined, self.Wz) + self.bz)
        combined_reset = torch.cat((rt * prev_h, xt), dim=1)
        h_tilde_t = torch.tanh(torch.matmul(combined_reset, self.Wh) + self.bh)
        ht = (1 - zt) * prev_h + zt * h_tilde_t
        return ht


class SimpleRNNLayer(nn.Module):
    def __init__(self, cell, input_size, hidden_size):
        super().__init__()
        self.cell = cell(input_size, hidden_size)
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.is_lstm = isinstance(self.cell, LSTMCellScratch)
    def forward(self, x, initial_state=None):
        batch_size, seq_len, _ = x.shape
        device = x.device
        if initial_state is None:
            h = torch.zeros(batch_size, self.hidden_size, device=device)
            c = torch.zeros(batch_size, self.hidden_size, device=device) if self.is_lstm else None
        else:
            if self.is_lstm: h, c = initial_state
            else: h = initial_state; c = None
        outputs = []
        for t in range(seq_len):
            xt = x[:, t, :]
            if self.is_lstm:
                h, c = self.cell(xt, (h, c))
                outputs.append(h.unsqueeze(1))
            else:
                h = self.cell(xt, h)
                outputs.append(h.unsqueeze(1))
        outputs = torch.cat(outputs, dim=1)
        final_state = (h, c) if self.is_lstm else h
        return outputs, final_state


class minLSTM(Module):
    def __init__(self, dim, expansion_factor = 1., proj_out = None):
        super().__init__()
        dim_inner = int(dim * expansion_factor)
        # *** proj_out 기본값 결정 수정 (명시적으로 전달받도록) ***
        # proj_out = default(proj_out, expansion_factor != 1.) -> 아래처럼 변경
        self.proj_out_active = default(proj_out, expansion_factor != 1.)

        self.to_hidden_and_f_i_gate = Linear(dim, dim_inner * 3, bias = False)
        # *** proj_out_active 플래그 사용 ***
        self.to_out = Linear(dim_inner, dim, bias = False) if self.proj_out_active else Identity()

    def forward(self, x, prev_hidden = None, return_next_prev_hidden = False):
        seq_len = x.shape[1]
        hidden, f_gate_raw, i_gate_raw = self.to_hidden_and_f_i_gate(x).chunk(3, dim = -1)

        if seq_len == 1:
            # --- 순차 경로 (단일 타임 스텝) ---
            hidden_candidate = hidden # g() 제거됨
            f_gate = f_gate_raw.sigmoid()
            i_gate = i_gate_raw.sigmoid()
            f_gate_prime = f_gate / (f_gate + i_gate + 1e-8)
            i_gate_prime = i_gate / (f_gate + i_gate + 1e-8)
            if exists(prev_hidden):
                out = (prev_hidden * f_gate_prime) + (hidden_candidate * i_gate_prime)
            else:
                out = hidden_candidate * i_gate_prime
        else:
            # --- 병렬 경로 (스캔) ---
            diff = F.softplus(-f_gate_raw) - F.softplus(-i_gate_raw)
            log_f = -F.softplus(diff)
            log_i = -F.softplus(-diff)
            log_tilde_h = log_g(hidden) # 병렬 경로는 log_g 유지 (주의사항 참고)
            log_values = log_i + log_tilde_h
            if exists(prev_hidden):
                try: log_h_0 = prev_hidden.log()
                except Exception as e: log_h_0 = torch.zeros_like(log_values[:, 0:1, :])
                log_values = torch.cat((log_h_0, log_values), dim = 1)
                log_f = F.pad(log_f, (0, 0, 1, 0), value=0.)
            out = heinsen_associative_scan_log(log_f, log_values)
            if exists(prev_hidden):
                 out = out[:, 1:]

        next_prev_hidden = out[:, -1:]
        # *** 최종 출력 투사 적용 ***
        out = self.to_out(out)

        if not return_next_prev_hidden:
            return out
        return out, next_prev_hidden


class minGRU(Module):
    def __init__(self, dim, expansion_factor = 1., proj_out = None):
        super().__init__()
        dim_inner = int(dim * expansion_factor)
        # *** proj_out 기본값 결정 수정 (명시적으로 전달받도록) ***
        self.proj_out_active = default(proj_out, expansion_factor != 1.)

        self.to_hidden_and_gate = Linear(dim, dim_inner * 2, bias = False)
        # *** proj_out_active 플래그 사용 ***
        self.to_out = Linear(dim_inner, dim, bias = False) if self.proj_out_active else Identity()

    def forward(self, x, prev_hidden = None, return_next_prev_hidden = False):
        seq_len = x.shape[1]
        hidden, gate_raw = self.to_hidden_and_gate(x).chunk(2, dim = -1)

        if seq_len == 1:
            # --- 순차 경로 (단일 타임 스텝) ---
            hidden_candidate = hidden # g() 제거됨
            gate = gate_raw.sigmoid() # z_t
            if exists(prev_hidden):
                out = torch.lerp(prev_hidden, hidden_candidate, gate)
            else:
                out = hidden_candidate * gate
        else:
            # --- 병렬 경로 (스캔) ---
            log_coeffs = -F.softplus(gate_raw) # log(1 - z_t)
            log_z = -F.softplus(-gate_raw)    # log(z_t)
            log_tilde_h = log_g(hidden) # 병렬 경로는 log_g 유지 (주의사항 참고)
            log_values = log_z + log_tilde_h
            if exists(prev_hidden):
                try: log_h_0 = prev_hidden.log()
                except Exception as e: log_h_0 = torch.zeros_like(log_values[:, 0:1, :])
                log_values = torch.cat((log_h_0, log_values), dim = 1)
                log_coeffs = F.pad(log_coeffs, (0, 0, 1, 0), value=0.)
            out = heinsen_associative_scan_log(log_coeffs, log_values)
            if exists(prev_hidden):
                 out = out[:, 1:]

        next_prev_hidden = out[:, -1:]
        # *** 최종 출력 투사 적용 ***
        out = self.to_out(out)

        if not return_next_prev_hidden:
            return out
        return out, next_prev_hidden


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
input_dim = 10
hidden_dim = 32
seq_len = 20
batch_size = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {device}")
lstm_scratch_layer = SimpleRNNLayer(LSTMCellScratch, input_dim, hidden_dim).to(device)
gru_scratch_layer = SimpleRNNLayer(GRUCellScratch, input_dim, hidden_dim).to(device)
# 샘플 테스트용 min 모델 생성 시 proj_out 기본값 사용 (True가 됨)
min_lstm_model_sample = minLSTM(input_dim, expansion_factor=hidden_dim/input_dim).to(device)
min_gru_model_sample = minGRU(input_dim, expansion_factor=hidden_dim/input_dim).to(device)
print("\n--- 파라미터 수 비교 (샘플 테스트 모델 기준) ---")
print(f"LSTM 직접 구현 레이어 : {count_parameters(lstm_scratch_layer):,}")
print(f"GRU 직접 구현 레이어  : {count_parameters(gru_scratch_layer):,}")
print(f"minLSTM (proj_out=True) : {count_parameters(min_lstm_model_sample):,}") # 출력 투사 포함
print(f"minGRU  (proj_out=True) : {count_parameters(min_gru_model_sample):,}") # 출력 투사 포함

print("\n--- 샘플 데이터 통과 테스트 ---")
x_sample = torch.randn(batch_size, seq_len, input_dim).to(device)
out_lstm_scratch, state_lstm_scratch = lstm_scratch_layer(x_sample)
out_gru_scratch, state_gru_scratch = gru_scratch_layer(x_sample)
# 샘플 테스트에서는 proj_out=True인 모델 사용
out_min_lstm = min_lstm_model_sample(x_sample)
out_min_gru = min_gru_model_sample(x_sample)
print(f"LSTM Scratch 출력 Shape: {out_lstm_scratch.shape}")
print(f"GRU Scratch 출력 Shape : {out_gru_scratch.shape}")
print(f"minLSTM 출력 Shape (proj_out=True) : {out_min_lstm.shape}") # (..., input_dim)
print(f"minGRU 출력 Shape  (proj_out=True) : {out_min_gru.shape}") # (..., input_dim)


print("\n--- 간단한 학습 태스크 (패리티 예측) ---")
def generate_parity_data(batch_size, seq_len, device):
    data = torch.randint(0, 2, (batch_size, seq_len, 1), dtype=torch.float32).to(device)
    labels = (data.sum(dim=1) % 2).float().to(device)
    return data, labels

class ParityClassifier(nn.Module):
    def __init__(self, rnn_layer, hidden_dim):
        super().__init__()
        self.rnn = rnn_layer
        self.fc = nn.Linear(hidden_dim, 1)
    def forward(self, x):
        if isinstance(self.rnn, SimpleRNNLayer):
             outputs, final_state = self.rnn(x)
             final_hidden = final_state[0] if self.rnn.is_lstm else final_state
        else:
             # 학습 시에는 proj_out=False인 모델이 전달될 것임
             outputs = self.rnn(x, prev_hidden=None)
             # 이 경우 outputs의 마지막 차원은 hidden_dim이 됨
             final_hidden = outputs[:, -1, :]
        return self.fc(final_hidden)

# 학습 파라미터 설정
learning_rate = 0.005
num_epochs = 10 # 에폭 수 확인/조정
train_steps_per_epoch = 100
parity_input_dim = 1
hidden_dim = 32

# 패리티 예측용 모델 인스턴스 생성 (*** 수정됨: proj_out=False 명시 ***)
lstm_parity = ParityClassifier(SimpleRNNLayer(LSTMCellScratch, parity_input_dim, hidden_dim), hidden_dim).to(device)
gru_parity = ParityClassifier(SimpleRNNLayer(GRUCellScratch, parity_input_dim, hidden_dim), hidden_dim).to(device)
min_lstm_parity = ParityClassifier(minLSTM(parity_input_dim, expansion_factor=hidden_dim/parity_input_dim, proj_out=False), hidden_dim).to(device)
min_gru_parity = ParityClassifier(minGRU(parity_input_dim, expansion_factor=hidden_dim/parity_input_dim, proj_out=False), hidden_dim).to(device)

# 학습시킬 모델 딕셔너리
models = {
    "LSTM_Scratch": lstm_parity,
    "GRU_Scratch": gru_parity,
    "minLSTM": min_lstm_parity,
    "minGRU": min_gru_parity
}

# 손실 함수 정의
criterion = nn.BCEWithLogitsLoss()

results = {}

# 학습 루프 (내용 동일)
for name, model in models.items():
    print(f"\n{name} 학습 시작...")
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    start_time = time.time()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for epoch in range(num_epochs):
        epoch_loss = 0
        epoch_correct = 0
        epoch_samples = 0
        for _ in range(train_steps_per_epoch):
            optimizer.zero_grad()
            data, labels = generate_parity_data(batch_size, seq_len, device)
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # 필요시 클리핑
            optimizer.step()
            epoch_loss += loss.item() * data.size(0)
            preds = (outputs > 0).float()
            epoch_correct += (preds == labels).sum().item()
            epoch_samples += data.size(0)
        avg_epoch_loss = epoch_loss / epoch_samples
        avg_epoch_acc = epoch_correct / epoch_samples
        if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1: # 진행 및 마지막 결과 출력
            print(f"에폭 [{epoch+1}/{num_epochs}], 손실: {avg_epoch_loss:.4f}, 정확도: {avg_epoch_acc:.4f}")
        total_loss += epoch_loss
        total_correct += epoch_correct
        total_samples += epoch_samples
    end_time = time.time()
    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples
    duration = end_time - start_time
    results[name] = {"loss": avg_loss, "accuracy": avg_acc, "time": duration}
    print(f"{name} - 최종 평균 손실: {avg_loss:.4f}, 최종 평균 정확도: {avg_acc:.4f}, 소요 시간: {duration:.2f}s")

# --- 최종 결과 비교 ---
print("\n--- 학습 결과 요약 ---")
print("모델명           | 손실   | 정확도 | 시간(s)")
print("-----------------|--------|--------|--------")
for name, res in results.items():
    print(f"{name:<15} | {res['loss']:.4f} | {res['accuracy']:.4f} | {res['time']:<7.2f}")

사용 디바이스: cuda

--- 파라미터 수 비교 (샘플 테스트 모델 기준) ---
LSTM 직접 구현 레이어 : 5,504
GRU 직접 구현 레이어  : 4,128
minLSTM (proj_out=True) : 1,280
minGRU  (proj_out=True) : 960

--- 샘플 데이터 통과 테스트 ---
LSTM Scratch 출력 Shape: torch.Size([64, 20, 32])
GRU Scratch 출력 Shape : torch.Size([64, 20, 32])
minLSTM 출력 Shape (proj_out=True) : torch.Size([64, 20, 10])
minGRU 출력 Shape  (proj_out=True) : torch.Size([64, 20, 10])

--- 간단한 학습 태스크 (패리티 예측) ---

LSTM_Scratch 학습 시작...
에폭 [5/10], 손실: 0.6933, 정확도: 0.5034
에폭 [10/10], 손실: 0.6934, 정확도: 0.4930
LSTM_Scratch - 최종 평균 손실: 0.6933, 최종 평균 정확도: 0.5005, 소요 시간: 21.98s

GRU_Scratch 학습 시작...
에폭 [5/10], 손실: 0.6931, 정확도: 0.5083
에폭 [10/10], 손실: 0.6933, 정확도: 0.5014
GRU_Scratch - 최종 평균 손실: 0.6934, 최종 평균 정확도: 0.4998, 소요 시간: 19.82s

minLSTM 학습 시작...
에폭 [5/10], 손실: 0.6950, 정확도: 0.4972
에폭 [10/10], 손실: 0.6946, 정확도: 0.5030
minLSTM - 최종 평균 손실: 0.6945, 최종 평균 정확도: 0.5023, 소요 시간: 2.67s

minGRU 학습 시작...
에폭 [5/10], 손실: 0.6937, 정확도: 0.5069
에폭 [10/10], 손실: 0.6961, 정확도: 0.4997
minGRU - 최종 평균 손실: 0.

In [8]:
# --- 필수 라이브러리 임포트 ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, Identity, Module
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence # 패딩을 위해 필요

# *** 데이터셋 라이브러리 ***
from datasets import load_dataset # Hugging Face datasets 라이브러리 임포트
# *** 어휘 구축용 ***
from collections import Counter, OrderedDict # OrderedDict는 안정적인 매핑 순서 보장

import torchinfo # 모델 구조 시각화
import math
import time
import numpy as np
import os

# --- 헬퍼 함수들 (이전과 동일) ---
def exists(v): return v is not None
def default(v, d): return v if exists(v) else d
def logcumsumexp(x, dim: int):
    if hasattr(torch, "logcumsumexp"): return torch.logcumsumexp(x, dim=dim)
    else:
        max_val = torch.max(x, dim=dim, keepdim=True).values
        max_val = torch.where(torch.isinf(max_val), torch.zeros_like(max_val), max_val)
        x_adjusted = x - max_val
        cumulative_exp = torch.cumsum(x_adjusted.exp(), dim=dim)
        log_cumulative_exp = torch.log(cumulative_exp.clamp(min=1e-38))
        return max_val + log_cumulative_exp
def heinsen_associative_scan_log(log_coeffs, log_values):
    log_coeffs = log_coeffs.contiguous(); log_values = log_values.contiguous()
    a_star = log_coeffs.cumsum(dim = 1)
    log_h0_plus_b_star = logcumsumexp(log_values - a_star, dim = 1)
    log_h = a_star + log_h0_plus_b_star
    return log_h.exp()
def g(x): return torch.where(x >= 0, x + 0.5, x.sigmoid())
def log_g(x):
    relu_x_plus_0_5 = F.relu(x) + 0.5
    return torch.where(x >= 0, relu_x_plus_0_5.log(), -F.softplus(-x))

# --- RNN Cell 및 Layer 구현 (이전과 동일) ---
# LSTMCellScratch, GRUCellScratch, SimpleRNNLayer, minLSTM, minGRU 클래스 정의
# ... (이전 답변의 클래스 정의 코드를 여기에 붙여넣으세요) ...
class LSTMCellScratch(nn.Module):
    """직접 구현한 LSTM Cell (PyTorch nn.Module 상속)"""
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        concat_size = input_size + hidden_size
        # 가중치 및 편향 파라미터 정의
        self.Wf = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bf = nn.Parameter(torch.Tensor(hidden_size))
        self.Wi = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bi = nn.Parameter(torch.Tensor(hidden_size))
        self.Wc = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bc = nn.Parameter(torch.Tensor(hidden_size))
        self.Wo = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bo = nn.Parameter(torch.Tensor(hidden_size))
        self.reset_parameters() # 파라미터 초기화 호출

    def reset_parameters(self):
        """파라미터 초기화"""
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
             nn.init.uniform_(weight, -stdv, stdv) # 균등 분포 초기화

    def forward(self, xt, prev_state):
        """LSTM Cell 단일 스텝 순전파"""
        prev_h, prev_c = prev_state # 이전 상태 분리
        combined = torch.cat((prev_h, xt), dim=1) # 입력과 이전 은닉 상태 결합
        # 게이트 및 상태 계산 (수식대로)
        ft = torch.sigmoid(torch.matmul(combined, self.Wf) + self.bf)
        it = torch.sigmoid(torch.matmul(combined, self.Wi) + self.bi)
        c_tilde_t = torch.tanh(torch.matmul(combined, self.Wc) + self.bc)
        ct = (ft * prev_c) + (it * c_tilde_t) # 셀 상태 업데이트
        ot = torch.sigmoid(torch.matmul(combined, self.Wo) + self.bo)
        ht = ot * torch.tanh(ct) # 은닉 상태 업데이트
        return ht, ct # 현재 상태 반환

class GRUCellScratch(nn.Module):
    """직접 구현한 GRU Cell (PyTorch nn.Module 상속)"""
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        concat_size = input_size + hidden_size
        # 가중치 및 편향 파라미터 정의
        self.Wr = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.br = nn.Parameter(torch.Tensor(hidden_size))
        self.Wz = nn.Parameter(torch.Tensor(concat_size, hidden_size))
        self.bz = nn.Parameter(torch.Tensor(hidden_size))
        self.Wh = nn.Parameter(torch.Tensor(concat_size, hidden_size)) # 후보 상태 계산용
        self.bh = nn.Parameter(torch.Tensor(hidden_size))
        self.reset_parameters() # 파라미터 초기화 호출

    def reset_parameters(self):
        """파라미터 초기화"""
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
             nn.init.uniform_(weight, -stdv, stdv) # 균등 분포 초기화

    def forward(self, xt, prev_h):
        """GRU Cell 단일 스텝 순전파"""
        combined = torch.cat((prev_h, xt), dim=1) # 입력과 이전 은닉 상태 결합
        # 게이트 및 상태 계산 (수식대로)
        rt = torch.sigmoid(torch.matmul(combined, self.Wr) + self.br) # 리셋 게이트
        zt = torch.sigmoid(torch.matmul(combined, self.Wz) + self.bz) # 업데이트 게이트
        # 후보 상태 계산 시 리셋 게이트 적용
        combined_reset = torch.cat((rt * prev_h, xt), dim=1)
        h_tilde_t = torch.tanh(torch.matmul(combined_reset, self.Wh) + self.bh)
        ht = (1 - zt) * prev_h + zt * h_tilde_t # 은닉 상태 업데이트
        return ht # 현재 은닉 상태 반환

class SimpleRNNLayer(nn.Module):
    """직접 구현한 Cell을 사용하여 시퀀스를 처리하는 레이어"""
    def __init__(self, cell, input_size, hidden_size):
        super().__init__()
        # 주어진 Cell 클래스로 인스턴스 생성
        self.cell = cell(input_size, hidden_size)
        self.input_size = input_size
        self.hidden_size = hidden_size
        # Cell 타입이 LSTM인지 확인 (상태 처리 방식 구분용)
        self.is_lstm = isinstance(self.cell, LSTMCellScratch)

    def forward(self, x, initial_state=None):
        """시퀀스 데이터 순전파"""
        batch_size, seq_len, _ = x.shape # 입력 shape 가져오기
        device = x.device # 입력 데이터의 디바이스 가져오기

        # 초기 상태 설정 (없으면 0으로 초기화)
        if initial_state is None:
            h = torch.zeros(batch_size, self.hidden_size, device=device)
            c = torch.zeros(batch_size, self.hidden_size, device=device) if self.is_lstm else None
        else: # 초기 상태가 주어지면 사용
            if self.is_lstm:
                h, c = initial_state
            else:
                h = initial_state
                c = None

        outputs = [] # 각 타임 스텝의 출력을 저장할 리스트
        # 시퀀스 길이에 대해 반복하며 Cell 실행
        for t in range(seq_len):
            xt = x[:, t, :] # 현재 타임 스텝의 입력
            if self.is_lstm: # LSTM Cell 처리
                h, c = self.cell(xt, (h, c))
                outputs.append(h.unsqueeze(1)) # 시간 차원 추가하여 저장
            else: # GRU Cell 처리
                h = self.cell(xt, h)
                outputs.append(h.unsqueeze(1)) # 시간 차원 추가하여 저장

        # 모든 타임 스텝의 출력을 하나의 텐서로 결합
        outputs = torch.cat(outputs, dim=1) # shape: (batch, seq_len, hidden)
        # 최종 상태 반환
        final_state = (h, c) if self.is_lstm else h
        return outputs, final_state

# --- 업데이트된 minLSTM 구현 ---
class minLSTM(Module):
    """Minimal LSTM (초기화 및 편향 추가)"""
    def __init__(self, dim, expansion_factor = 1., proj_out = None):
        super().__init__()
        dim_inner = int(dim * expansion_factor)
        self.proj_out_active = default(proj_out, expansion_factor != 1.)

        # *** 수정: bias=True 로 변경 ***
        self.to_hidden_and_f_i_gate = Linear(dim, dim_inner * 3, bias = True)
        self.to_out = Linear(dim_inner, dim, bias = True) if self.proj_out_active else Identity()

        # *** 추가: 파라미터 초기화 호출 ***
        self.reset_parameters()

    # *** 추가: 파라미터 초기화 메소드 ***
    def reset_parameters(self):
        # 예시: Xavier 초기화 (다른 방식도 가능)
        for name, param in self.named_parameters():
            if param.dim() > 1 : # 가중치 행렬
                nn.init.xavier_uniform_(param)
            elif "bias" in name: # 편향 벡터
                 nn.init.zeros_(param)


    def forward(self, x, prev_hidden = None, return_next_prev_hidden = False):
        # forward 메소드 내용은 이전과 동일
        seq_len = x.shape[1]; device = x.device
        hidden, f_gate_raw, i_gate_raw = self.to_hidden_and_f_i_gate(x).chunk(3, dim = -1)
        if seq_len == 1:
            hidden_candidate = hidden
            f_gate = f_gate_raw.sigmoid(); i_gate = i_gate_raw.sigmoid()
            f_gate_prime = f_gate / (f_gate + i_gate + 1e-8); i_gate_prime = i_gate / (f_gate + i_gate + 1e-8)
            if exists(prev_hidden): out = (prev_hidden * f_gate_prime) + (hidden_candidate * i_gate_prime)
            else: out = hidden_candidate * i_gate_prime
        else:
            diff = F.softplus(-f_gate_raw) - F.softplus(-i_gate_raw)
            log_f = -F.softplus(diff); log_i = -F.softplus(-diff)
            log_tilde_h = log_g(hidden) # 병렬 경로는 log_g 유지
            log_values = log_i + log_tilde_h
            if exists(prev_hidden):
                try: log_h_0 = prev_hidden.log()
                except Exception as e: log_h_0 = torch.zeros((x.shape[0], 1, hidden.shape[-1]), device=device)
                log_values = torch.cat((log_h_0, log_values), dim = 1); log_f = F.pad(log_f, (0, 0, 1, 0), value=0.)
            out = heinsen_associative_scan_log(log_f, log_values)
            if exists(prev_hidden): out = out[:, 1:]
        next_prev_hidden = out[:, -1:]
        out = self.to_out(out)
        if not return_next_prev_hidden: return out
        return out, next_prev_hidden

# --- 업데이트된 minGRU 구현 ---
class minGRU(Module):
    """Minimal GRU (초기화 및 편향 추가)"""
    def __init__(self, dim, expansion_factor = 1., proj_out = None):
        super().__init__()
        dim_inner = int(dim * expansion_factor)
        self.proj_out_active = default(proj_out, expansion_factor != 1.)

        # *** 수정: bias=True 로 변경 ***
        self.to_hidden_and_gate = Linear(dim, dim_inner * 2, bias = True)
        self.to_out = Linear(dim_inner, dim, bias = True) if self.proj_out_active else Identity()

        # *** 추가: 파라미터 초기화 호출 ***
        self.reset_parameters()

    # *** 추가: 파라미터 초기화 메소드 ***
    def reset_parameters(self):
        # 예시: Xavier 초기화
        for name, param in self.named_parameters():
            if param.dim() > 1 : # 가중치 행렬
                nn.init.xavier_uniform_(param)
            elif "bias" in name: # 편향 벡터
                 nn.init.zeros_(param)

    def forward(self, x, prev_hidden = None, return_next_prev_hidden = False):
        # forward 메소드 내용은 이전과 동일
        seq_len = x.shape[1]; device = x.device
        hidden, gate_raw = self.to_hidden_and_gate(x).chunk(2, dim = -1)
        if seq_len == 1:
            hidden_candidate = hidden
            gate = gate_raw.sigmoid()
            if exists(prev_hidden): out = torch.lerp(prev_hidden, hidden_candidate, gate)
            else: out = hidden_candidate * gate
        else:
            log_coeffs = -F.softplus(gate_raw); log_z = -F.softplus(-gate_raw)
            log_tilde_h = log_g(hidden) # 병렬 경로는 log_g 유지
            log_values = log_z + log_tilde_h
            if exists(prev_hidden):
                try: log_h_0 = prev_hidden.log()
                except Exception as e: log_h_0 = torch.zeros((x.shape[0], 1, hidden.shape[-1]), device=device)
                log_values = torch.cat((log_h_0, log_values), dim = 1); log_coeffs = F.pad(log_coeffs, (0, 0, 1, 0), value=0.)
            out = heinsen_associative_scan_log(log_coeffs, log_values)
            if exists(prev_hidden): out = out[:, 1:]
        next_prev_hidden = out[:, -1:]
        out = self.to_out(out)
        if not return_next_prev_hidden: return out
        return out, next_prev_hidden


# === 데이터 로딩 및 전처리 (Hugging Face datasets + 수동 전처리) ===
print("\n--- AG_NEWS 데이터 로딩 및 전처리 (수동 토크나이저/단어집합) ---")

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {device}")

# 데이터셋 로드
try:
    ag_news_dataset = load_dataset("ag_news")
    print("AG_NEWS 데이터셋 로드 완료.")
except Exception as e:
    print(f"Hugging Face datasets 로드 중 오류: {e}")
    exit()

# --- 토크나이저 (기본 split 사용) ---
tokenizer = lambda x: x.lower().split()
print("기본 토크나이저 (str.lower().split()) 사용")

# --- 단어 집합(Vocabulary) 수동 구축 ---
print("단어 집합 구축 시작...")
counter = Counter()
# 학습 데이터셋 텍스트를 순회하며 단어 빈도 계산
for example in ag_news_dataset['train']:
    counter.update(tokenizer(example['text']))

# 최소 빈도수 적용 및 정렬
min_freq = 5
specials = ["<unk>", "<pad>"]
# OrderedDict를 사용하여 인덱스<->단어 매핑 순서 고정
itos = OrderedDict()
# 특수 토큰 먼저 추가
for i, s in enumerate(specials):
    itos[i] = s
# 단어 추가 (빈도수 높은 순)
curr_idx = len(specials)
word_freq = sorted(counter.items(), key=lambda x: x[1], reverse=True)
for word, freq in word_freq:
    if freq >= min_freq:
        itos[curr_idx] = word
        curr_idx += 1

# 단어 -> 인덱스 매핑 생성
stoi = {s: i for i, s in itos.items()}
vocab_size = len(itos)
unk_idx = stoi["<unk>"]
pad_idx = stoi["<pad>"]
print(f"단어 집합 크기 (min_freq={min_freq}): {vocab_size}")
print(f"Unk 인덱스: {unk_idx}, Pad 인덱스: {pad_idx}")
print("단어 집합 구축 완료.")

# 텍스트/레이블 파이프라인 정의 (수동 vocab 사용)
text_pipeline = lambda x: [stoi.get(token, unk_idx) for token in tokenizer(x)]
label_pipeline = lambda x: int(x) # AG_NEWS 레이블 (0-3)

# 전처리 함수 (datasets.map 용)
def preprocess_function(examples):
    tokenized_texts = [text_pipeline(text) for text in examples['text']]
    labels = [label_pipeline(label) for label in examples['label']]
    return {'input_ids': tokenized_texts, 'label': labels}

# 데이터셋에 전처리 함수 적용
print("데이터셋 전처리 적용 중...")
# num_proc > 1 로 설정하여 병렬 처리 가능 (환경에 따라)
tokenized_datasets = ag_news_dataset.map(preprocess_function, batched=True, remove_columns=['text'])
print("데이터셋 전처리 완료.")

# DataLoader를 위한 Collate 함수
def collate_batch_manual(batch):
    label_list, text_list, lengths = [], [], []
    for item in batch: # batch는 dict 리스트 {'input_ids': [...], 'label': ...}
        label_list.append(item['label'])
        processed_text = torch.tensor(item['input_ids'], dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(len(processed_text))

    labels = torch.tensor(label_list, dtype=torch.int64)
    texts_padded = pad_sequence(text_list, batch_first=True, padding_value=pad_idx)
    lengths_tensor = torch.tensor(lengths, dtype=torch.int64) # 길이는 현재 사용 안 함
    return labels.to(device), texts_padded.to(device), lengths_tensor.to(device)

# DataLoader 생성
batch_size = 512
train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_batch_manual)
test_dataloader = DataLoader(tokenized_datasets['test'], batch_size=batch_size, shuffle=False, collate_fn=collate_batch_manual)
print("데이터 로더 생성 완료.")

# === 텍스트 분류 모델 정의 (TextClassifier - 이전과 동일, 단 pad_idx 전달) ===
class TextClassifier(nn.Module):
    """텍스트 분류 모델"""
    def __init__(self, vocab_size, embedding_dim, rnn_layer, hidden_dim, num_classes, pad_idx): # pad_idx 추가
        super().__init__()
        # 임베딩 레이어에 padding_idx 지정
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = rnn_layer
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.hidden_dim = hidden_dim
        self.rnn_is_custom_layer = isinstance(self.rnn, SimpleRNNLayer)
        if not self.rnn_is_custom_layer:
             self.rnn_is_min_model = isinstance(self.rnn, (minLSTM, minGRU))
        else:
             self.rnn_is_min_model = False

    def forward(self, text, lengths=None): # lengths 인자 추가 (현재 사용 안 함)
        embedded = self.embedding(text)

        if self.rnn_is_custom_layer:
             outputs, final_state = self.rnn(embedded)
             final_hidden = final_state[0] if self.rnn.is_lstm else final_state
        elif self.rnn_is_min_model:
             outputs = self.rnn(embedded, prev_hidden=None)
             final_hidden = outputs[:, -1, :]
        else: # 다른 RNN 타입 대비
             outputs, (final_hidden, _) = self.rnn(embedded)
             final_hidden = final_hidden[-1]

        return self.fc(final_hidden)

# === 모델 구조 시각화 (torchinfo 사용 - 이전과 동일) ===
print("\n--- 모델 구조 시각화 (minLSTM 예시) ---")
# 파라미터 설정
embedding_dim = 64
hidden_dim = 128
num_classes = 4

# 시각화용 모델 인스턴스 생성 (minLSTM 사용, proj_out=False)
temp_rnn = minLSTM(embedding_dim, expansion_factor=hidden_dim/embedding_dim, proj_out=False).to(device)
# TextClassifier에 pad_idx 전달
model_for_info = TextClassifier(vocab_size, embedding_dim, temp_rnn, hidden_dim, num_classes, pad_idx).to(device)

# 더미 입력 데이터 생성
dummy_batch_size = 4
dummy_seq_len = 50
dummy_input_ids = torch.randint(0, vocab_size, (dummy_batch_size, dummy_seq_len), dtype=torch.long).to(device)

# torchinfo.summary 실행
try:
    torchinfo.summary(model_for_info, input_data=(dummy_input_ids,),
                      col_names=["input_size", "output_size", "num_params", "mult_adds"],
                      row_settings=["var_names"])
except Exception as e:
    print(f"torchinfo.summary 실행 중 오류: {e}")

# === 학습 및 평가 (이전과 동일) ===
print("\n--- 텍스트 분류 학습 및 평가 ---")

# 학습 파라미터
learning_rate = 0.001
num_epochs = 20

# 모델 인스턴스 생성 (TextClassifier 사용, pad_idx 전달)
lstm_clf = TextClassifier(vocab_size, embedding_dim,
                         SimpleRNNLayer(LSTMCellScratch, embedding_dim, hidden_dim),
                         hidden_dim, num_classes, pad_idx).to(device)
gru_clf = TextClassifier(vocab_size, embedding_dim,
                        SimpleRNNLayer(GRUCellScratch, embedding_dim, hidden_dim),
                        hidden_dim, num_classes, pad_idx).to(device)
min_lstm_clf = TextClassifier(vocab_size, embedding_dim,
                             minLSTM(embedding_dim, expansion_factor=hidden_dim/embedding_dim, proj_out=False),
                             hidden_dim, num_classes, pad_idx).to(device)
min_gru_clf = TextClassifier(vocab_size, embedding_dim,
                            minGRU(embedding_dim, expansion_factor=hidden_dim/embedding_dim, proj_out=False),
                            hidden_dim, num_classes, pad_idx).to(device)

models_clf = {
    "LSTM_Scratch": lstm_clf,
    "GRU_Scratch": gru_clf,
    "minLSTM": min_lstm_clf,
    "minGRU": min_gru_clf
}

# 손실 함수
criterion = nn.CrossEntropyLoss() # 패딩은 임베딩 레이어에서 처리되므로 무시 불필요

results_clf = {}

# 정확도 계산 함수 (이전과 동일)
def calculate_accuracy(outputs, labels):
    preds = outputs.argmax(dim=1)
    correct = (preds == labels).sum().item()
    return correct / len(labels) if len(labels) > 0 else 0.0

# 학습 및 평가 함수 (이전과 동일)
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0; total_acc = 0; total_samples = 0
    for i, (labels, text, lengths) in enumerate(dataloader):
        labels, text = labels.to(device), text.to(device)
        optimizer.zero_grad()
        outputs = model(text) # lengths 인자 제거
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        batch_samples = labels.size(0)
        total_loss += loss.item() * batch_samples
        total_acc += calculate_accuracy(outputs, labels) * batch_samples
        total_samples += batch_samples
        # if (i + 1) % 100 == 0: print(f"  스텝 [{i+1}/{len(dataloader)}], 손실: {total_loss/total_samples:.4f}, 정확도: {total_acc/total_samples:.4f}") # 너무 자주 출력될 수 있음
        del labels, text, lengths, outputs, loss
        if torch.cuda.is_available(): torch.cuda.empty_cache()
    return total_loss / total_samples, total_acc / total_samples

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0; total_acc = 0; total_samples = 0
    with torch.no_grad():
        for labels, text, lengths in dataloader:
            labels, text = labels.to(device), text.to(device)
            outputs = model(text) # lengths 인자 제거
            loss = criterion(outputs, labels)
            batch_samples = labels.size(0)
            total_loss += loss.item() * batch_samples
            total_acc += calculate_accuracy(outputs, labels) * batch_samples
            total_samples += batch_samples
            del labels, text, lengths, outputs, loss
            if torch.cuda.is_available(): torch.cuda.empty_cache()
    return total_loss / total_samples, total_acc / total_samples

# 학습 루프 실행 (이전과 동일)
for name, model in models_clf.items():
    print(f"\n===== {name} 학습 시작 =====")
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    start_time = time.time()
    best_val_acc = 0
    for epoch in range(num_epochs):
        print(f"--- 에폭 {epoch+1}/{num_epochs} ---")
        train_loss, train_acc = train_epoch(model, train_dataloader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, test_dataloader, criterion, device)
        if val_acc > best_val_acc: best_val_acc = val_acc
        print(f"에폭 [{epoch+1}/{num_epochs}] 결과 | Train 손실: {train_loss:.4f} | Train 정확도: {train_acc:.4f} | Val 손실: {val_loss:.4f} | Val 정확도: {val_acc:.4f}")
    end_time = time.time()
    duration = end_time - start_time
    final_val_loss, final_val_acc = evaluate(model, test_dataloader, criterion, device)
    results_clf[name] = {"loss": final_val_loss, "accuracy": final_val_acc, "time": duration}
    print(f"===== {name} 학습 완료 =====")
    print(f"{name} - 최종 검증 손실: {final_val_loss:.4f}, 최종 검증 정확도: {final_val_acc:.4f}, 총 소요 시간: {duration:.2f}s")

# --- 최종 결과 비교 ---
print("\n--- 학습 결과 요약 (AG_NEWS) ---")
print("모델명           | 최종 검증 손실 | 최종 검증 정확도 | 시간(s)")
print("-----------------|----------------|-----------------|--------")
for name, res in results_clf.items():
    print(f"{name:<15} | {res['loss']:.4f}         | {res['accuracy']:.4f}          | {res['time']:<7.2f}")



--- AG_NEWS 데이터 로딩 및 전처리 (수동 토크나이저/단어집합) ---
사용 디바이스: cuda
AG_NEWS 데이터셋 로드 완료.
기본 토크나이저 (str.lower().split()) 사용
단어 집합 구축 시작...
단어 집합 크기 (min_freq=5): 39546
Unk 인덱스: 0, Pad 인덱스: 1
단어 집합 구축 완료.
데이터셋 전처리 적용 중...
데이터셋 전처리 완료.
데이터 로더 생성 완료.

--- 모델 구조 시각화 (minLSTM 예시) ---

--- 텍스트 분류 학습 및 평가 ---

===== LSTM_Scratch 학습 시작 =====
--- 에폭 1/20 ---
에폭 [1/20] 결과 | Train 손실: 1.3857 | Train 정확도: 0.2513 | Val 손실: 1.3856 | Val 정확도: 0.2514
--- 에폭 2/20 ---
에폭 [2/20] 결과 | Train 손실: 1.3853 | Train 정확도: 0.2521 | Val 손실: 1.3855 | Val 정확도: 0.2499
--- 에폭 3/20 ---
에폭 [3/20] 결과 | Train 손실: 1.3850 | Train 정확도: 0.2520 | Val 손실: 1.3855 | Val 정확도: 0.2514
--- 에폭 4/20 ---
에폭 [4/20] 결과 | Train 손실: 1.3874 | Train 정확도: 0.2532 | Val 손실: 1.3856 | Val 정확도: 0.2509
--- 에폭 5/20 ---
에폭 [5/20] 결과 | Train 손실: 1.3850 | Train 정확도: 0.2514 | Val 손실: 1.3853 | Val 정확도: 0.2513
--- 에폭 6/20 ---
에폭 [6/20] 결과 | Train 손실: 1.3855 | Train 정확도: 0.2507 | Val 손실: 1.3856 | Val 정확도: 0.2496
--- 에폭 7/20 ---
에폭 [7/20] 결과 | Train 손실: 1.3846 | Train 

`모델의 구조를 보다 뚜렷하게 명시(레이어 수 등)하고, 전체적인 레이어 구조를 맞추는 경우와 파라미터 수를 맞추는 경우에 대해서도 제대로 된 데이터로 비교 필요`